In [0]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [0]:
mnist_df, mnist_info = tfds.load('mnist',with_info=True,as_supervised=True)

In [0]:
mnist_train, mnist_test = mnist_df['train'], mnist_df['test']
num_val_samples = 0.1 * mnist_info.splits['train'].num_examples
num_val_samples = tf.cast(num_val_samples, tf.int64)
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [0]:
def scale(img, label):
  img = tf.cast(img, tf.float32)
  img /= 255.
  return img,label

scaled_train_and_val_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

In [0]:
BUFFER_SIZE = 10000
shuffled_train_and_val_data = scaled_train_and_val_data.shuffle(BUFFER_SIZE)
test_data = scaled_test_data.shuffle(BUFFER_SIZE)

In [0]:
val_data = shuffled_train_and_val_data.take(num_val_samples)
train_data = shuffled_train_and_val_data.skip(num_val_samples)

In [0]:
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
val_data = val_data.batch(num_val_samples)
test_data = test_data.batch(num_test_samples)

val_inp, val_tar = next(iter(val_data))

In [0]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
])

In [0]:
adam = tf.keras.optimizers.Adam(
    learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)

model.compile(optimizer=adam,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [28]:
NUM_EPOCHS = 5
model.fit(train_data, epochs=NUM_EPOCHS,validation_data=(val_inp,val_tar), verbose=2)

Epoch 1/5
540/540 - 11s - loss: 0.0795 - accuracy: 0.9773 - val_loss: 0.0801 - val_accuracy: 0.9782
Epoch 2/5
540/540 - 11s - loss: 0.0749 - accuracy: 0.9788 - val_loss: 0.0749 - val_accuracy: 0.9797
Epoch 3/5
540/540 - 11s - loss: 0.0711 - accuracy: 0.9801 - val_loss: 0.0717 - val_accuracy: 0.9792
Epoch 4/5
540/540 - 11s - loss: 0.0673 - accuracy: 0.9810 - val_loss: 0.0680 - val_accuracy: 0.9808
Epoch 5/5
540/540 - 11s - loss: 0.0634 - accuracy: 0.9821 - val_loss: 0.0661 - val_accuracy: 0.9823


In [29]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 1ms/step - loss: 0.0871 - accuracy: 0.9726
